In [1]:
try:
    import transformers
except ImportError as e:
    print('transformers not installed')
    print('Installing now...')
    !pip install -q git+https://github.com/huggingface/transformers.git
    print("Install complete.")
    pass  

In [2]:
import torch
import io 
import os
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.metrics import classification_report,accuracy_score
import transformers
import json
from tqdm.notebook import tqdm
from transformers.utils.dummy_pt_objects import AutoModelForSequenceClassification
from transformers import AutoModelForTokenClassification,AutoConfig,AutoModel,AutoTokenizer,BertModel,BertConfig,AdamW, get_constant_schedule,BertForSequenceClassification,get_linear_schedule_with_warmup
import random
import numpy as np
import torch.nn as nn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

#Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# if using Google Colab, set colab = True
colab = False

if colab == True:
    #Mounting Drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    
    %cd '/content/gdrive/Shareddrives/523 Project/Data'
    %ls
else:
    DATA_DIR = '/projectnb2/dl523/students/kjv/EC523_Project/Data/'

In [4]:
#Reading in headlines data
df = pd.read_json(DATA_DIR + "News Headlines/Sarcasm_Headlines_Dataset_v2.json",lines = True)
df = df.rename(columns={'is_sarcastic': 'label'})
df = df.drop('article_link', 1)
df.head()

#splits for headlines training, test, and validation

train_headlines, temporary_text, train_label, temporary_label = train_test_split(df['headline'], df['label'], 
                                                                    random_state=200, 
                                                                    test_size=0.2, 
                                                                    stratify=df['label'])







validation_headlines, test_headlines, validation_label, test_label = train_test_split(temporary_text, temporary_label, 
                                                                    random_state=200, 
                                                                    test_size=0.5, 
                                                                    stratify=temporary_label)

In [6]:
# set max length for padding/clipping during tokenization
max_length = 35

# create tokenized training, validation, and test splits
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

training_tokens = tokenizer.__call__(train_headlines.tolist(),max_length = max_length,padding = True,truncation = True)
validation_tokens = tokenizer.__call__(validation_headlines.tolist(),max_length = max_length,padding = True,truncation = True)
test_tokens = tokenizer.__call__(test_headlines.tolist(),max_length = max_length,padding= True,truncation = True)

# Stacking the inputs as tensors for use in the BERT model

training_set = TensorDataset(torch.tensor(training_tokens['input_ids']),torch.tensor(training_tokens['attention_mask']),torch.tensor(train_label.tolist()))
validation_set = TensorDataset(torch.tensor(validation_tokens['input_ids']),torch.tensor(validation_tokens['attention_mask']),torch.tensor(validation_label.tolist()))
test_set = TensorDataset(torch.tensor(test_tokens['input_ids']),torch.tensor(test_tokens['attention_mask']),torch.tensor(test_label.tolist()))

In [17]:
class multihead_attn_bert(nn.Module):

    def __init__(self, bert_encoder, embed_dim, num_attn_layers, num_heads):
        super(multihead_attn_bert, self).__init__()
        
        self.bert = bert_encoder
        
        self.multiheads = nn.ModuleList([nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)])
        self.multiheads.extend([nn.MultiheadAttention(embed_dim, num_heads, batch_first=True) for i in range(num_attn_layers-1)])
        
        self.GRU = nn.GRU(input_size=embed_dim, hidden_size=512, bidirectional=True, batch_first=True)
        
        # 1024 = hidden_size of GRU x 2 (for bidirectionality of GRU)
        self.fc = nn.Linear(in_features=1024, out_features=1)
        
        # use dropout set of 0.2 as in paper
        self.dropout = nn.Dropout(p=0.2)
        
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokenized_input_values, attention_mask):
        
        output = self.bert(tokenized_input_values, attention_mask=attention_mask).last_hidden_state
        
        for multihead_layer in self.multiheads:
            output,_ = multihead_layer(query=output,key=output,value=output,key_padding_mask=(~attention_mask.bool()))
        
        _,hidden = self.GRU(output)
        
        # concatenate bidirectional outputs from GRU to pass to linear layer
        hidden = torch.cat([hidden[0,:, :], hidden[1,:,:]], dim=1).unsqueeze(0)
        
        output = self.fc(hidden)
        
        output = self.dropout(output)
        
        output = self.sigmoid(output)
        
        return output

In [8]:
# initialize pre-trained BERT large

bertconfig = BertConfig()
bert = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# freeze pre-trained layers in BERT

for param in bert.parameters():
    param.requires_grad = False

In [16]:
# initialize multihead attention sarcasm model with BERT embedder

# embed_dim = 768 if using bert_base, 1024 for bert_large
mh_sarcasm_model = multihead_attn_bert(bert, embed_dim=768, num_attn_layers=3, num_heads=8)
mh_sarcasm_model.to(device)

# save untrained model weights
torch.save(mh_sarcasm_model.state_dict(), "/projectnb/dl523/students/kjv/EC523_Project/Saved_Models/Multihead_BERT/untrained_mhbert.pth")

multihead_attn_bert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [12]:
#Dataloaders for the headlines sets

batch_size = 64

trainloader = DataLoader(training_set, batch_size = batch_size, num_workers=2, shuffle = True)
validationloader = DataLoader(validation_set, batch_size = batch_size, num_workers=2, shuffle = True)
testloader = DataLoader(test_set, batch_size = batch_size, num_workers=2, shuffle = True)

#Loss function (used in reference multihead attention paper)
criterion = nn.BCELoss()

In [20]:
# define training, testing, and validation loss functions

def train_mh_bert(model, trainloader, validationloader, optimizer, criterion, num_epochs, scheduler=None):
        
        # conditions for early stopping
        last_val_loss = float('inf')
        min_val_loss = float('inf')
        patience = 3
        es_counter = 0
        
        print("Starting training...")
        
        for epoch in range(1, num_epochs+1):
    
            model.train()
            if scheduler != None:
                print("Learning rate: ", scheduler.get_last_lr())
            
            running_loss = 0
            print('Epoch: ',epoch)

            for idx, (inputs,attention_mask,label) in enumerate(tqdm(trainloader,total = len(trainloader))):

                inputs, attention_mask, label = inputs.to(device),attention_mask.to(device),label.to(device)

                optimizer.zero_grad()

                output = mh_sarcasm_model(inputs, attention_mask)
                output = torch.flatten(output)

                # convert label type from int to float for use in BCELoss
                label = label.float()
                loss = criterion(output,label)

                loss.backward()
                running_loss += loss.item()
                optimizer.step()

                # print loss every 100 batches
                if idx % 100 == 0:
                    print('Loss: ',float(loss))
                    running_loss = 0.0
            
            # adjust scheduler after every epoch
            if scheduler != None:
                scheduler.step()

            # check for changes in avg validation loss to determine if early stopping is needed
            print("Checking validation loss...")
            curr_val_loss = validation_loss(model, validationloader)
            print("Average validation loss after last epoch: ", curr_val_loss)

            if curr_val_loss > last_val_loss:
                es_counter += 1

                if es_counter >= patience:
                    print("Early stopping triggered. Ending training..")
                    return
                else:
                    print(f"Increase in validation loss! {patience-es_counter} more consecutive loss increase(s) until early stop.")

            else:
                print("Decrease in validation loss. Early stop counter reset to 0.")
                es_counter = 0

            last_val_loss = curr_val_loss
            
            # check to save model if validation loss is lower than min recorded validation loss
            if curr_val_loss < min_val_loss:
                min_val_loss = curr_val_loss
                save_dir = "/projectnb/dl523/students/kjv/EC523_Project/Saved_Models/Multihead_BERT/trained_MH_BERT.pth"
                torch.save(model.state_dict(), save_dir)

def test_mh_bert(model, testloader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in testloader:
            
            inputs, attn_mask, labels = data
            inputs, attn_mask, labels = inputs.to(device), attn_mask.to(device), labels.to(device)
            
            output = torch.flatten(model(inputs, attn_mask))
            
            # convert output to class predictions
            output[output<0.5] = 0
            output[output>=0.5] = 1
            
            total += labels.size(0)
            correct += (output==labels).float().sum().item()
            
        acc = correct/total * 100
    
    return acc

def validation_loss(model, validationloader):
    model.eval()
    total_val_loss = 0
    criterion = nn.BCELoss()
    
    with torch.no_grad():
        for data in validationloader:
            
            inputs, attn_mask, labels = data
            inputs, attn_mask, labels = inputs.to(device), attn_mask.to(device), labels.to(device)
            labels = labels.float()
            
            output = torch.flatten(model(inputs, attn_mask))
            loss = criterion(output, labels)
            
            total_val_loss += loss.item()
    
    avg_val_loss = total_val_loss/len(validationloader)
    
    return avg_val_loss

In [18]:
#Training mh_sarcasm bert
num_epochs = 50

# optimizer using learning rate from reference paper
tuning_parameters = [parameter for parameter in mh_sarcasm_model.parameters() if parameter.requires_grad]
optimizer = torch.optim.Adam(tuning_parameters,lr = 1e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps = 0,num_training_steps = len(trainloader)*Epochs)

# train the model with early stopping (add scheduler once done)
train_mh_bert(mh_sarcasm_model, trainloader, validationloader, optimizer, criterion, num_epochs)

Starting training...
Epoch:  1


  0%|          | 0/358 [00:00<?, ?it/s]

Loss:  0.6963216066360474
Loss:  0.418545663356781
Loss:  0.2411714792251587
Loss:  0.42320960760116577
Checking validation loss...
Average validation loss after last epoch:  0.2985598643620809
Decrease in validation loss. Early stop counter reset to 0.
Epoch:  2


  0%|          | 0/358 [00:00<?, ?it/s]

Loss:  0.2823479473590851
Loss:  0.27974405884742737
Loss:  0.3039320111274719
Loss:  0.25664085149765015
Checking validation loss...
Average validation loss after last epoch:  0.2584587554136912
Decrease in validation loss. Early stop counter reset to 0.
Epoch:  3


...

Loss:  0.043110959231853485
Loss:  0.0023991051129996777
Loss:  0.038173235952854156
Loss:  0.06937780976295471
Checking validation loss...
Average validation loss after last epoch:  0.38699375010199016
Increase in validation loss! 2 more consecutive loss increase(s) until early stop.
Epoch:  49


  0%|          | 0/358 [00:00<?, ?it/s]

Loss:  0.0008595343679189682
Loss:  0.051669199019670486
Loss:  0.07512842863798141
Loss:  0.0026416971813887358
Checking validation loss...
Average validation loss after last epoch:  0.3472749236557219
Decrease in validation loss. Early stop counter reset to 0.
Epoch:  50


  0%|          | 0/358 [00:00<?, ?it/s]

Loss:  0.0034973248839378357
Loss:  0.07195261120796204
Loss:  0.030228853225708008
Loss:  0.030221175402402878
Checking validation loss...
Average validation loss after last epoch:  0.38445756344331633
Increase in validation loss! 2 more consecutive loss increase(s) until early stop.


In [19]:
# test mh_sarcasm_bert

acc = test_mh_bert(mh_sarcasm_model, testloader)
print(f"Accuracy of network: {acc}")

Accuracy of network: 91.61425576519916


In [ ]:
# hp validation for mh_bert

def perform_hp_tuning(model, trainingset, validationset, lr_array, batchsize_array, criterion, num_epochs=1):

    accuracies = np.empty([len(lr_array), len(batchsize_array)])
    tuning_parameters = [parameter for parameter in model.parameters() if parameter.requires_grad]

    # run validation testing

    for i, lr in enumerate(lr_array):
        
        # reset optimizer with new learning rate
        optimizer = optim.SGD(tuning_parameters, lr=lr, momentum = .9)

        for j, batch_size in enumerate(batchsize_array):

            print("LEARNING RATE: ", lr)
            print("BATCH SIZE: ", batch_size)

            # restore untrained weights for model
            model.load_state_dict(torch.load("/projectnb/dl523/students/kjv/EC523_Project/Saved_Models/Multihead_BERT/untrained_mhbert.pth"))

            # define data loaders for training and testing data
            trainloader = torch.utils.data.DataLoader(trainingset, batch_size=batch_size, num_workers=2)
            testloader = torch.utils.data.DataLoader(validationset, batch_size=batch_size, num_workers=2)
            
            # train model using current hps and early stopping
            train_mh_bert(model, trainloader, testloader, optimizer, criterion, num_epochs)
            
            # test performance on validation dataset
            result = test_mh_bert(model, testloader)

            accuracies[i,j] = result
            print(f"Accuracy for lr={lr} and bs={batch_size}: {accuracies[i,j]}\n")


    # choose learning rate and batch size with best validation accuracy
    print("---HP TESTING COMPLETE---")
    print("Accuracy Matrix: \n", accuracies)
    best_lr_ind, best_bs_ind = np.unravel_index(np.argmax(accuracies, axis=None), accuracies.shape)
    
    optimal_lr = learning_rates[best_lr_ind]
    optimal_batch_size = batchsize_array[best_bs_ind]

    print(f"\nBest learning rate: {optimal_lr}")
    print(f"Best batch size: {optimal_batch_size}")
    return optimal_lr, optimal_batch_size